<a href="https://colab.research.google.com/github/TimL2/Group_Project_2_ML_Bot/blob/Shay/ML_Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install alpaca-trade-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Initial imports
import pandas as pd
import alpaca_trade_api as tradeapi

In [3]:
ALPACA_API_KEY = 'PKETI0Q5C8PPUXVNHJFJ'
ALPACA_SECRET_KEY = 'WQxmEpBCoMXydCCg0G8cUe6hGLpgaAfrvedZ09Fy'
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

In [4]:
# Create a connection to the API 
api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_API_BASE_URL, api_version="v2")

In [5]:
# Set signal variable
signal = 1

In [6]:
 # Create buy signal, num shares and ticker
if signal == 1:
    orderSide = "buy"
else:
    orderSide = "sell"

In [7]:
# Set the ticket symbol and the number of shares to buy
ticker = "TSLA"
number_of_shares = 1

In [8]:
 # Make API call
prices = api.get_bars(ticker, "1Min").df

# Reorganize the DataFrame
prices = pd.concat([prices], axis=1, keys=["TSLA"])

# Get final closing price
limit_amount = prices["TSLA"]["close"][-1]

In [9]:
prices

TSLA                                     \
                             open      high     low     close volume   
timestamp                                                              
2022-10-20 08:00:00+00:00  209.74  209.7400  208.41  209.2800  22663   
2022-10-20 08:01:00+00:00  209.15  209.2000  207.64  207.6700  25749   
2022-10-20 08:02:00+00:00  208.00  208.8300  208.00  208.5300  12136   
2022-10-20 08:03:00+00:00  208.71  208.9500  208.67  208.9500   7855   
2022-10-20 08:04:00+00:00  208.95  209.2000  208.82  208.8500  18764   
...                           ...       ...     ...       ...    ...   
2022-10-20 23:52:00+00:00  206.40  206.4000  206.33  206.4000   1852   
2022-10-20 23:53:00+00:00  206.40  206.4599  206.40  206.4599   2916   
2022-10-20 23:54:00+00:00  206.45  206.5900  206.45  206.5900   4160   
2022-10-20 23:55:00+00:00  206.60  206.7000  206.60  206.7000   3578   
2022-10-20 23:56:00+00:00  206.75  206.7500  206.65  206.6500   2584   

                                                   
                          trade_count        vwap  
timestamp                                          
2022-10-20 08:00:00+00:00         621  209.105361  
2022-10-20 08:01:00+00:00        1001  208.255293  
2022-10-20 08:02:00+00:00         475  208.401262  
2022-10-20 08:03:00+00:00         275  208.743472  
2022-10-20 08:04:00+00:00         570  208.977297  
...                               ...         ...  
2022-10-20 23:52:00+00:00          78  206.361380  
2022-10-20 23:53:00+00:00          52  206.422773  
2022-10-20 23:54:00+00:00          69  206.490134  
2022-10-20 23:55:00+00:00          78  206.618524  
2022-10-20 23:56:00+00:00          95  206.679346  

[944 rows x 7 columns]

In [10]:
limit_amount

206.65

# *SVC MODEL**

In [11]:
# Calculate the daily returns using the closing prices and the pct_change function
prices["actual_returns"] = prices['TSLA']["close"].pct_change()

# Display sample data
prices.head()


TSLA                                             \
                             open    high     low   close volume trade_count   
timestamp                                                                      
2022-10-20 08:00:00+00:00  209.74  209.74  208.41  209.28  22663         621   
2022-10-20 08:01:00+00:00  209.15  209.20  207.64  207.67  25749        1001   
2022-10-20 08:02:00+00:00  208.00  208.83  208.00  208.53  12136         475   
2022-10-20 08:03:00+00:00  208.71  208.95  208.67  208.95   7855         275   
2022-10-20 08:04:00+00:00  208.95  209.20  208.82  208.85  18764         570   

                                      actual_returns  
                                 vwap                 
timestamp                                             
2022-10-20 08:00:00+00:00  209.105361            NaN  
2022-10-20 08:01:00+00:00  208.255293      -0.007693  
2022-10-20 08:02:00+00:00  208.401262       0.004141  
2022-10-20 08:03:00+00:00  208.743472       0.002014  
2022-10-20 08:04:00+00:00  208.977297      -0.000479

In [12]:
# Drop all NaN values from the DataFrame
prices = prices.dropna()

# Review the DataFrame
display(prices.head())
display(prices.tail())

TSLA                                             \
                             open    high     low   close volume trade_count   
timestamp                                                                      
2022-10-20 08:01:00+00:00  209.15  209.20  207.64  207.67  25749        1001   
2022-10-20 08:02:00+00:00  208.00  208.83  208.00  208.53  12136         475   
2022-10-20 08:03:00+00:00  208.71  208.95  208.67  208.95   7855         275   
2022-10-20 08:04:00+00:00  208.95  209.20  208.82  208.85  18764         570   
2022-10-20 08:05:00+00:00  208.85  209.37  208.85  209.33   5353         215   

                                      actual_returns  
                                 vwap                 
timestamp                                             
2022-10-20 08:01:00+00:00  208.255293      -0.007693  
2022-10-20 08:02:00+00:00  208.401262       0.004141  
2022-10-20 08:03:00+00:00  208.743472       0.002014  
2022-10-20 08:04:00+00:00  208.977297      -0.000479  
2022-10-20 08:05:00+00:00  209.103850       0.002298

TSLA                                     \
                             open      high     low     close volume   
timestamp                                                              
2022-10-20 23:52:00+00:00  206.40  206.4000  206.33  206.4000   1852   
2022-10-20 23:53:00+00:00  206.40  206.4599  206.40  206.4599   2916   
2022-10-20 23:54:00+00:00  206.45  206.5900  206.45  206.5900   4160   
2022-10-20 23:55:00+00:00  206.60  206.7000  206.60  206.7000   3578   
2022-10-20 23:56:00+00:00  206.75  206.7500  206.65  206.6500   2584   

                                                  actual_returns  
                          trade_count        vwap                 
timestamp                                                         
2022-10-20 23:52:00+00:00          78  206.361380       0.000000  
2022-10-20 23:53:00+00:00          52  206.422773       0.000290  
2022-10-20 23:54:00+00:00          69  206.490134       0.000630  
2022-10-20 23:55:00+00:00          78  206.618524       0.000532  
2022-10-20 23:56:00+00:00          95  206.679346      -0.000242

In [13]:
# Define a window size of 9
short_window = 4

# Create an SMA that uses short_window, and assign it to a new column named “sma_fast”
prices["sma_fast"] = prices['TSLA']["close"].rolling(window=short_window).mean()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
# Define a window size of 20
long_window = 20

# Create an SMA that uses long_window, and assign it to a new columns named “sma_slow”
prices["sma_slow"] = prices['TSLA']["close"].rolling(window=short_window).mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
prices = prices.dropna()

In [16]:
# Assign a copy of the `sma_fast` and `sma_slow` columns to a new DataFrame called `X`
X = prices[["sma_fast", "sma_slow"]].shift().dropna().copy()

# Display sample data
display(X.head())
display(X.tail())

,sma_fast,sma_slow
,,
timestamp,,
2022-10-20 08:05:00+00:00,208.5000,208.5000
2022-10-20 08:06:00+00:00,208.9150,208.9150
2022-10-20 08:07:00+00:00,209.1675,209.1675
2022-10-20 08:08:00+00:00,209.3550,209.3550
2022-10-20 08:09:00+00:00,209.6475,209.6475


,sma_fast,sma_slow
,,
timestamp,,
2022-10-20 23:52:00+00:00,206.325000,206.325000
2022-10-20 23:53:00+00:00,206.337500,206.337500
2022-10-20 23:54:00+00:00,206.389975,206.389975
2022-10-20 23:55:00+00:00,206.462475,206.462475
2022-10-20 23:56:00+00:00,206.537475,206.537475


In [17]:
# Create a new column in the `prices` called "signal" setting its value to zero.
prices["signal"] = 0.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
# Create the signal to sell
prices.loc[(prices["actual_returns"] < 0), "signal"] = -1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [19]:
# Copy the new "signal" column to a new Series called `y`.
y = prices["signal"].copy()

In [20]:
# Import required libraries
from pandas.tseries.offsets import DateOffset

In [21]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2022-10-20 08:05:00+00:00


In [22]:
# Select the ending period for the training data with an offset of 18 months
training_end = X.index.min() + DateOffset(months=18)

# Display the training end date
print(training_end)

2024-04-20 08:05:00+00:00


In [23]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [24]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [25]:
# Import required libraries
from sklearn.preprocessing import StandardScaler

In [26]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


ValueError: ignored

In [ ]:
# Import the SVM model
from sklearn import svm
from sklearn.metrics import classification_report

In [ ]:
# Create the classifier model
svm_model = svm.SVC()

# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

In [ ]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display report
print(training_report)

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [ ]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

In [ ]:
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

predictions_df["predicted_signal"] = testing_signal_predictions

predictions_df["actual_returns"] = trading_df["actual_returns"]

predictions_df["trading_algorithm_returns"] = (
    predictions_df["actual_returns"] * predictions_df["predicted_signal"]
)

# Review the DataFrame
predictions_df.head()

In [ ]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().plot()